# Bi-Directional JavaScript - Python

**Note:** This notebook does currently not work in jupyterlab, as [javascript execution is disabled](https://github.com/jupyterlab/jupyterlab/issues/3118) at the moment. Hopefully [this PR](https://github.com/jupyterlab/jupyterlab/pull/4515) will resolve the issue. Once the PR finds it way into a release I will have another look at jupyterlab.

Each widget has "model" that allwow to exchange messages between the python process and the javascript process.

In [1]:
import json
import random
import time

import ipywidgets
from traitlets import Unicode

from IPython.display import display

## JavaScript to Python

This section summarized essentially how the button is internally implemented. Its implementation can be found

- [here](https://github.com/jupyter-widgets/ipywidgets/blob/master/packages/controls/src/widget_button.ts) for its JavaScript part and
- [here](https://github.com/jupyter-widgets/ipywidgets/blob/master/ipywidgets/widgets/widget_button.py) for its Python part.

In [2]:
class JavaScriptToPython(ipywidgets.DOMWidget):
    _view_name = Unicode('JavaScriptToPython').tag(sync=True)
    _view_module = Unicode('javascript2python').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

In [3]:
%%javascript
require.undef('javascript2python');

define('javascript2python', ["@jupyter-widgets/base"], function(widgets) {
    const JavaScriptToPython = widgets.DOMWidgetView.extend({
        render: function() {
            const button = document.createElement('button');
            button.innerHTML = 'click me';
            button.addEventListener('click', () => this.send('clicked at '+ new Date));
            
            this.el.appendChild(button);
        },
    });
    return {JavaScriptToPython};
});

<IPython.core.display.Javascript object>

In [4]:
widget = JavaScriptToPython()

@widget.on_msg
def _(widget, content, buffers):
    print(content)

display(widget)

JavaScriptToPython()

## Python to JavaScript

Any message send via `widget.send(...)` from python can be received with `this.model.on('msg:custom', ...)` in JavaScript.

In [5]:
class PythonToJavaScript(ipywidgets.DOMWidget):
    _view_name = Unicode('PythonToJavaScript').tag(sync=True)
    _view_module = Unicode('python2javascript').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

In [6]:
%%javascript
require.undef('python2javascript');

define('python2javascript', ["@jupyter-widgets/base"], function(widgets) {
    const PythonToJavaScript = widgets.DOMWidgetView.extend({
        render: function() {
            this.model.on('msg:custom', data => {    
                this.el.innerHTML = JSON.stringify(data);
            });
        },
    });
    return {PythonToJavaScript};
});

<IPython.core.display.Javascript object>

In [7]:
widget = PythonToJavaScript()
display(widget)

# NOTE: the widget is not yet rendered, event will be 'lost'
widget.send({'foo': 'bar'})

PythonToJavaScript()

In [8]:
# NOTE: the widget is already rendered, the event will be received
widget.send({'hello': 'world'})

Since the widget only starts to listend to events once rendered, the call to `widget.send` has to be in a new cell.

An alternative would be to use the widget properties (similar to the [example in the docs](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html#Updating-a-widget-in-the-background)). However, this approach would mean to always keep the full state on both sides. With streaming data, this may inccur high transfer costs.

## Example live updates for Leaflet maps

Implementin a leaflet map that supports liveupdates is a bit more complicated, since it requires using an iframe to embedd the required JavaScript and CSS resources.

Luckily, the [`postMessage` API](https://developer.mozilla.org/en-US/docs/Web/API/Window/postMessage) to send messages between the current browser window, the Jupyter frontend, and the embedded iframe, the map.

To test the API, the following examples sends messages between iframe and its parent whenever one of the buttons is clicked.

In [9]:
%%javascript
// an example using postMessage to communicate with an iframe

(element => {
const iframeSource = `
<button id="my-button"></button>
<script>
(() => {
    const targetOrigin = "${window.origin}";

    const button = document.getElementById('my-button');
    button.dataset.count = 0;
    button.innerHTML = 'Clicks from Jupyter: 0';

    button.addEventListener('click', () => {
        window.parent.postMessage('increment', targetOrigin);
    });

    window.addEventListener('message', ev => {
        if(ev.origin != targetOrigin) {
            return;
        }
        console.log('foo');
        button.dataset.count++;
        button.innerHTML = 'Clicks from Jupyter: ' + button.dataset.count;
    });
})();
</script>
`;

const button = document.createElement('button');
button.dataset.count = 0;
button.innerHTML = 'Clicks from IFrame: 0';

button.addEventListener('click', () => {
    // TODO: what is the correct origin of the iframe?
    iframe.contentWindow.postMessage('increment', '*');
});

const iframe = document.createElement('iframe');
iframe.width = 200;
iframe.height = 50;
iframe.src = "data:text/html;base64," + btoa(iframeSource);

element[0].appendChild(button);
element[0].appendChild(iframe);

const messageListener = ev => {
    // TODO: is this really the correct origin?
    if(ev.origin != "null") {
        return;
    }
    button.dataset.count++;
    button.innerHTML = 'Clicks from IFrame: ' + button.dataset.count;
}

// Make sure to remove the listener when the element is removed
// For simplicity use the  JQuery remove event
element.on(
    'remove', 
    () => window.removeEventListener('message', messageListener),
);

window.addEventListener("message", messageListener);
})(element);

<IPython.core.display.Javascript object>

The live-updating leaflet map will combine the `postMessage` event systems with jupyter's event system. First the update is send from python to the widget, which will forward the message into the iframe, where the leaflet map is updated.

In [10]:
class LiveLeaflet(ipywidgets.DOMWidget):
    _view_name = Unicode('LiveLeaflet').tag(sync=True)
    _view_module = Unicode('liveleaflet').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

In [11]:
%%javascript

(element => {
require.undef('liveleaflet');

const iframeSource = `
<html>
<head>
<link 
    rel="stylesheet" 
    href="https://unpkg.com/leaflet@1.3.1/dist/leaflet.css"
    integrity="sha512-Rksm5RenBEKSKFjgI3a41vrjkw4EVPlJ3+OiI65vTjIdo9brlAacEuKOiQ5OFh7cOI1bkDwLqdLw3Zg0cRJAAQ=="
    crossorigin=""/>
<script 
    src="https://unpkg.com/leaflet@1.3.1/dist/leaflet.js"
    integrity="sha512-/Nsx9X4HebavoBvEBuyp3I7od5tA0UzAxs+j83KgC8PU0kgB4XiK4Lfe4y4cgBtaRJQEIFCW+oC506aPT2L1zw=="
    crossorigin=""></script>
</head>
<body>
<div id="mapid" style="height: 400px; width: 400px;"></div>
<script>
(function() {
    var mymap = L.map("mapid").setView([48.14, 11.57], 10);
    L.tileLayer("http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png").addTo(mymap);
    
    var marker = L.marker([48.14, 11.57]).addTo(mymap);
    
    window.onmessage = function(msg) {
        var ev = msg["data"];
        
        if(ev["type"] == "update") {
            marker.setLatLng([ev["y"], ev["x"]]);
        }
        else {
            console.log("unknown event", ev);
        }
    }
})();
</script>
</body>
</html>
`

define('liveleaflet', ["@jupyter-widgets/base"], function(widgets) {
    
    const LiveLeaflet = widgets.DOMWidgetView.extend({
        render: function() {
            const iframe = document.createElement('iframe');
            iframe.width = 420;
            iframe.height = 420;
            iframe.src = "data:text/html;base64," + btoa(iframeSource);
            this.el.appendChild(iframe);
            
            // forward all message from python into the iframe
            this.model.on('msg:custom', function(data) {
               iframe.contentWindow.postMessage(data, '*');
            });
        },
    });
    
    return {LiveLeaflet};
});
    
})(element);

<IPython.core.display.Javascript object>

In [12]:
widget = LiveLeaflet()
widget

LiveLeaflet()

In [13]:
# perform a random walk of the marker
y, x = 48.14, 11.57
for _ in range(10):
    x += random.normalvariate(0, 0.01)
    y += random.normalvariate(0, 0.01)
    
    widget.send(dict(type='update', x=x, y=y))
    time.sleep(1)

## Live Altair / Vega Charts

In [14]:
import altair as alt
from vegawidget import VegaWidget

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [15]:
data = alt.InlineData([
    dict(c=0, v=1, t=1),
    dict(c=1, v=2, t=2),
    dict(c=2, v=3, t=3),
    dict(c=0, v=4, t=4),
    dict(c=1, v=5, t=5),
    dict(c=0, v=6, t=6),
    dict(c=0, v=7, t=7),
    dict(c=1, v=8, t=8),
    dict(c=2, v=9, t=9),
])

chart = alt.Chart(data).mark_bar().encode(x='c:O', y='sum(v):Q')

In [16]:
# update the spec to include a data name
# TODO: once on PyPI use InlineData(name='...')
spec = chart.to_dict()
spec['data']['name'] = 'table'

widget = VegaWidget(spec=spec)
display(widget)

VegaWidget(spec_source='{"config": {"view": {"width": 400, "height": 300}}, "data": {"values": [{"c": 0, "v": …

In [17]:
# update the plot dynamically
for t in range(10, 20):
    value = dict(c=random.choice([0, 1, 2]), v=random.randint(1, 10), t=t)
    widget.update('table', remove=f'datum.t < {t - 5}', insert=[value])
    time.sleep(1)